In [1]:
import env
import acquire
import numpy as np
import pandas as pd

Acquire data from mySQL using the python module to connect and query. 
**Objective: create acquire.py**

- single dataframe

- content requirements:
> logerror
>
> all fields
>
> airconditioningtypeid
>
> all properties having transaction in 2017, 
>
> include only the last transaction for each properity = no duplicate property id's
>
> zestimate error 
>
> date of transaction
> 
> only include properties that include a latitude and longitude value

In [2]:
df = acquire.get_data_from_mysql()
df = acquire.basic_clean(df)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77381 entries, 17052889 to 14447871
Data columns (total 64 columns):
county                          77381 non-null object
state                           77381 non-null object
logerror                        77381 non-null float64
transactiondate                 77381 non-null object
id                              77381 non-null int64
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingqualitytypeid           49672 non-null float64
calculatedbathnbr               76772 non-null float64
decktypeid                      614 non-null float64
finishedfloor1squarefeet        6023 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
finishedsquarefeet12            73749 non-null float64
finishedsquarefeet13            41 non-null float64
finishedsquarefeet15            3009 non-null float64
finishedsquarefe

Summarize data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [20]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    pct_missing = num_missing/rows
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'pct_rows_missing': pct_missing}).sort_values(by = 'pct_rows_missing', ascending = False)
    return cols_missing

In [24]:
def nulls_by_row(df):
    num_cols_missing = df.isnull().sum(axis=1)
    pct_cols_missing = df.isnull().sum(axis=1)/df.shape[1]*100
    rows_missing = pd.DataFrame({'num_cols_missing': num_cols_missing,\
                                 'pct_cols_missing': pct_cols_missing})\
                                .reset_index()\
                                .groupby(['num_cols_missing','pct_cols_missing'])\
                                .count()\
                                .rename(index=str, columns={'index': 'num_rows'}).reset_index()
    return rows_missing

In [3]:
def df_value_counts(df):
    counts = pd.Series([])
    for i, col in enumerate(df.columns.values):
        if df[col].dtype == 'object':
            col_count = df[col].value_counts()
        else:
            col_count = df[col].value_counts(bins=10, sort=False)
        counts = counts.append(col_count)
    return counts


def sam_df_value_counts(df):
    for col in df.columns:
        print(f'{col}:')
        if df[col].dtype == 'object':
            col_count = df[col].value_counts()
        else:
            if df[col].nunique() >= 35:
                col_count = df[col].value_counts(bins=10, sort=False)
            else:
                col_count = df[col].value_counts()
        print(col_count)

- Keep only single unit properties. (e.g. no duplexes, no land/lot, ...)
> not only filter unitcnt
>
> consider logics like: bed#, sqft, unit type...etc.
>
> identify those with unitcnt not defined

create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

The input:
a dataframe
a number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing)
a number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. i.e. if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing)
The output:
the dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
hint:
look up the dropna documentation
you will want to compute a threshold from your input values (prop_required) and total number of rows or columns
make use of inplace